## 1.任务描述
在医学搜索中，对搜索问题的意图分类可以极大提升搜索结果的相关性，特别是医学知识具备极强的专业性，对问题意图进行分类也有助于融入医学知识来做增强搜索结果的性能。本任务数据集就是在这样的背景下产生的。

## 2.任务说明
在本次评测中，医学问题分为 病情诊断(diagnosis）、病因分析(cause)、治疗方案(method)、就医建议(advice)、指标解读(metric_explain)、疾病描述(disease_express)、后果表述(result)、注意事项(attention)、功效作用(effect)、医疗费用(price)、其他(other) 共11种类型，类型说明和示例如下：

- 病情诊断：已知症状，判断可能的原因， 如：

    - 最近早上起来浑身无力是怎么回事？
    - 我家宝宝快五个月了，为什么偶尔会吐清水带？
- 病因分析：已知疾病，解释疾病发生的原因。如：

    - 阴道松弛的原因是什么？
    - 鼻咽癌是如何发生的？
- 治疗方案：已知疾病/症状，给出治疗或缓解的方案（检查/手术/药物/行为）。如：

    - 腰椎间盘突出可以烤电吗
    - 感冒头疼吃什么药好
    - 宝宝感冒眼屎多又黄怎么办
    - 烫伤的疤痕要怎么去除？
- 就医建议：已知症状/疾病，给出就医建议（科室/检查）。如：

    - 糖尿病该做什么检查？
    - 肚子疼去什么科室？
- 指标解读：身高/体重/血压等检查结果的数值范围解读。如：

    - 血常规超敏C反应蛋白偏高说明什么
    - b超检查报告写的检测到盆腔积液是11mm，严重么？
- 疾病描述：疾病属性（eg：能不能治、能不能治好）、症状、表现、图片等相关表述。如：

    - 外痔疮早期症状有哪些呢？
    - 白癜风能不能治愈
- 后果表述：疾病/症状/药品/检查项/食物的危害，疾病恶化不治疗会产生的不良影响或治疗后会产生好的结果。如：

    - 缺乏钾元素会怎么样
    - 乙肝不治疗会怎么样
- 注意事项：病人要注意的事情，以及分析食物的好坏，食物对病人的影响。如：

    - 哮喘应该注意些什么
    - 孕妇能不能吃榴莲
    - 柿子不能和什么一起吃
    - 糖尿病人饮食注意什么啊？
- 功效作用：食品/药物的好处，功效/作用/副作用。如：

    - 乌鸡白凤丸的功效和作用
    - 玫瑰，柠檬，菊花可以一起泡吗？有什么功效
- 医疗费用：疾病/手术/药品/检查/的费用。如：

    - 二甲双瓜要多少钱？
- 其他：无法涵盖在前面分类里的以及低价值/无意义/非医疗、需求不明没讲明白的。如：

    - 玻尿酸丰唇能保持多久
    - 血常规五分类是查什么
## 3.评测指标
本任务的评价指标使用准确率Accuracy来评估，即：

准确率(Accuracy) = #预测正确的条目数 / #预测总条目数

## 4.评测数据
本评测开放训练集数据6931条，验证集数据1955条，测试集数据1994条。

In [1]:
# cut
import pandas as pd
import jieba
import re

# stop_words=list()
#  stop_words = list(pd.read_csv(r'stopwords\total_stopwords.txt',header=None,sep='\\n',encoding='utf-8',engine='python')[0])
def tokenize(text):
#     remove_chars = '[。·’!"\#$%&\'()＃！（）*+,-./:;<=>?\@，：?￥★、…．＞【】［］《》？“”‘’\[\\]^_`{|}~]+'
#     text = re.sub(remove_chars, "", text)
#     cut = jieba.lcut(text.replace(" ",""))    
#     result = list()
#     for word in cut:   
#         if word not in stop_words:   
#             result.append(word)
    result = list(text)
    return result

## 说明
### 分词
- 这里的注释是最早对文本进行分词处理的方式：
    - 使用jieba分词并去除一定的停用词
- 但最后在所有任务中都采用了逐字分词的方案
    - 这是处于想要屏蔽分词效果的影响
    - 同时考虑dnn对于逐字处理可能具有一定优势
    - 再是当前这些任务的特殊背景（医疗文本）可能进行逐字处理有优势 
        - 一方面在专业术语和日常用语混杂的情况，错误的分词很可能出现，而且一旦出现就可能导致严重的误判
        - 另一方面，专业术语中有相当数量的生僻字，而这些生僻字往往可以是很重要的特征
            - 其次，这些生僻字数量又是相对有限的
            - 并且这些生僻字的顺序调换可能就会指代比较相似但不同的实体
    - 但还需要强调的是，以上内容是个人的主观感受，并没有特别严谨的实验验证
        - 指一次改了很多变量导致性能提升而没法确定到底哪个是好处哪个是坏处
    - 并且其实并不是每个任务以上思路都是适用的
        - 一个特别典型的例子就是CHIP-CDN
        - 该任务为术语标准化，其中第一个步骤为召回
            - 召回采用的是tdidf的特征矩阵加上Cluster pruning
            - 在召回步骤中，采用逐字分词完全可能导致非常严重的后果
                - 因为tdidf算法是只考虑词频而不考虑词序的，他是一个词袋模型
                - 而术语召回的过程中有大量词频十分类似，需要具体词语来判断的词语
                - 这就会导致在召回的过程就命中率不高
                - 而pipline的方案在非dnn的步骤就可能成为效果的瓶颈
        - 上述说明其实对于dnn可能也是适用的，由于dnn的特性而没法下断言
        - 并且虽然说了这么多其实也没有特别严谨的实验验证
### 其余
其余数据处理依旧是和别的任务一般大差不差

In [2]:
# word to sequence
UNK_TAG = "UNK"
PAD_TAG = "PAD"
class Word2Sequence():
    UNK = 0
    PAD = 1

    def __init__(self):
        self.word2index_dict = {
            UNK_TAG : self.UNK,
            PAD_TAG : self.PAD,
        }
        self.count = {}


    def fit(self, sentence):
        # 保存句子到dict
        for word in sentence:
            self.count[word] = self.count.get(word, 0) + 1
        

    def build_vocab(self,min=0,max=None,max_features=None):
        self.count = {word:value for word,value in self.count.items() if value > min}
        if(max is not None):
            self.count = {word:value for word,value in self.count.items() if value < max}
        if max_features is not None:
            self.count = dict(sorted(self.count.items(), key = lambda x:x[-1], reverse=True)[:max_features])

        for word in self.count:
            self.word2index_dict[word] = len(self.word2index_dict)
        self.index2word_dict = dict(zip(self.word2index_dict.values(), self.word2index_dict.keys()))


    def words2index_transform(self, sentence, max_len=None):
        if max_len is not None:
            if max_len > len(sentence):
                sentence = sentence + [PAD_TAG] * (max_len - len(sentence))
            else:
                sentence = sentence[:max_len]
        return [self.word2index_dict.get(word, self.UNK) for word in sentence]


    def index2words_transform(self, sentence):
        return [self.index2word_dict.get(index) for index in sentence]

    
    def __len__(self):
        return len(self.word2index_dict)

In [3]:
# dictionary build
import pickle
from tqdm import tqdm
import json
import os

train_data_path = r"data\KUAKE-QIC\KUAKE-QIC_train.json"
test_data_path = r"data\KUAKE-QIC\KUAKE-QIC_test.json"
dev_data_path = r"data\KUAKE-QIC\KUAKE-QIC_dev.json"
if(not os.path.exists("models/KUAKE-QIC_Word2Sequence.pkl")):
    word_index_tranformer = Word2Sequence()
    with open(train_data_path, encoding="utf-8") as f:
        for data in tqdm(json.load(f)):
            word_index_tranformer.fit(tokenize(data['query']))
    word_index_tranformer.build_vocab()
    pickle.dump(word_index_tranformer, open(r"models/KUAKE-QIC_Word2Sequence.pkl", 'wb'))
else:
    word_index_tranformer = pickle.load(open(r"models/KUAKE-QIC_Word2Sequence.pkl", 'rb'))
print('\n' + str(len(word_index_tranformer)))


2405


In [4]:
# label encode
import numpy as np
import torch
labels_name = """病情诊断
病因分析
治疗方案
就医建议
指标解读
疾病表述
后果表述
注意事项
功效作用
医疗费用
其他"""
label2num_dict = {line:index for index, line in enumerate(labels_name.split('\n'))} 
num2label_dict = {index:line for index, line in enumerate(labels_name.split('\n'))} 
print(label2num_dict)
print(num2label_dict)

{'病情诊断': 0, '病因分析': 1, '治疗方案': 2, '就医建议': 3, '指标解读': 4, '疾病表述': 5, '后果表述': 6, '注意事项': 7, '功效作用': 8, '医疗费用': 9, '其他': 10}
{0: '病情诊断', 1: '病因分析', 2: '治疗方案', 3: '就医建议', 4: '指标解读', 5: '疾病表述', 6: '后果表述', 7: '注意事项', 8: '功效作用', 9: '医疗费用', 10: '其他'}


In [5]:
# dataset
from torch.utils.data import Dataset
import json


max_sentece_length = 20
class RosDataset(Dataset):
    def __init__(self, data_path, train=True):
        self.train = train
        with open(data_path, encoding="utf-8") as f:
            self.data_list = json.load(f)

    def __getitem__(self, index):
        # 获取索引对应位置的一条数据
        cuted_text = tokenize(self.data_list[index]["query"])
        indexed_text = torch.LongTensor(word_index_tranformer.words2index_transform(cuted_text, max_len=max_sentece_length))
        if(self.train):
            label = label2num_dict[self.data_list[index]["label"]]
            return label, indexed_text
        else:
            return indexed_text

    def __len__(self):
        # 返回数据的总数量
        return len(self.data_list)

train_dataset = RosDataset(train_data_path)
dev_dataset = RosDataset(dev_data_path)
test_dataset = RosDataset(test_data_path, train=False)
print(train_dataset[0])
print(len(train_dataset))
print(test_dataset[0])

(2, tensor([2, 3, 4, 5, 6, 7, 8, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))
6931
tensor([ 599, 1345,    0, 1208,    4,  344,   99,  617,   60,   95, 1230,  283,
          95,    5,    6,    1,    1,    1,    1,    1])


In [6]:
# dataloader
from torch.utils.data import DataLoader
import torch

max_sentece_length = 20
def collate_fn(batch):
    #  batch是一个列表，其中是一个一个的元组，每个元组是dataset中_getitem__的结果
    word_index_tranformer = pickle.load(open(r"models/KUAKE-QIC_Word2Sequence.pkl", 'rb'))
    batch = list(zip(*batch))
    return torch.LongTensor(batch[0]), torch.stack(batch[1])


train_data_loader = DataLoader(dataset=train_dataset,batch_size=128,shuffle=True,collate_fn=collate_fn)
dev_data_loader = DataLoader(dataset=dev_dataset,batch_size=1,shuffle=True,collate_fn=collate_fn)
test_data_loader = DataLoader(dataset=test_dataset,batch_size=1,shuffle=False, drop_last=False)
for index, text in enumerate(test_data_loader):
    if(index > 0):
        break
    print(f"{index}:{text}")

0:tensor([[ 599, 1345,    0, 1208,    4,  344,   99,  617,   60,   95, 1230,  283,
           95,    5,    6,    1,    1,    1,    1,    1]])


In [7]:
import tqdm
def train(epochs, model, model_path=None, optimizer_path=None, device=None):
    model = model.to(device)
    model.train()
    optimizer = Adam(model.parameters(), lr=0.001)
    for epoch in tqdm(range(epochs), desc="Train"):
        for index, (label, text) in enumerate(train_data_loader):
            if not device is None:
                label = label.to(device)
                text = text.to(device)
            optimizer.zero_grad()
            output = model(text)
            loss = F.nll_loss(output, label)
            loss.backward()
            optimizer.step()
    
    if not model_path is None:
        torch.save(model.state_dict(), model_path)
    if not optimizer_path is None:
        torch.save(optimizer.state_dict(), optimizer_path)


def evaluation_accuracy(model, test_data_loader, device=None):
    count_correct = 0
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        for label, text in tqdm(dev_data_loader, desc="Evaluation"):
            if not device is None:
                label = label.to(device)
                text = text.to(device)
            if(model(text).argmax() == label):
                count_correct = count_correct + 1
    print(f"\n{count_correct}/{len(test_data_loader)}")
    return count_correct / len(test_data_loader)

In [8]:
# lstm
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim import Adam

class BiLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(len(word_index_tranformer), 300)
        self.bilstm = nn.LSTM(input_size=300, num_layers=3, hidden_size=128, batch_first=True,bidirectional=True, dropout=0.5)
        self.fc = nn.Linear(128*2, 11)


    def forward(self, input):
        x = self.embedding(input)
        # x:[batch_size, max_len, 2*hidden_size]
        # h_n:[2*2, batch_size, hidden_size]
        # c_n:[2*2, batch_size, hidden_size]
        x, (h_n, c_n) = self.bilstm(x)
        forward = h_n[-1, :, :]
        backward = h_n[-2, :, :]
        # output:[batch_size, hidden_size*2]
        output = torch.cat([forward, backward], dim=-1)
        output = self.fc(output)
        return F.log_softmax(output, dim=-1)

train_mode = True
bilstm_model = BiLSTM()
model_path = "models\QIC_bilstm.pth"
optimizer_path = "models\QIC_bilstm_optim.pth"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if(train_mode):
    train(epochs=40, model=bilstm_model, model_path="models\QIC_bilstm.pth", optimizer_path=optimizer_path, device=device)
else:
    bilstm_model.load_state_dict(torch.load(model_path))

print('\n' + str(evaluation_accuracy(bilstm_model, dev_data_loader, device=device)))

Evaluation: 100%|██████████| 1955/1955 [00:13<00:00, 146.40it/s]
1367/1955

0.69923273657289



## BiLSTM model
- embedding
- BiLSTM
- FC
## LSTM Structure
![LSTM](resrc\LSTM.png)
## BiRNN Structure
![BiRNN](resrc\BiRNN.png)
## SRNN Structure
![SRNN](resrc\SRNN.png)
## 说明
- 模型非常的简单，只是做了embedding之后送到3层双向LSTM中后经过全连接层分类

In [9]:
dump_file_path = "result\KUAKE-QIC_test.json"
with open(test_data_path,'r',encoding="utf-8") as source:
    data = json.load(source)
    bilstm_model = bilstm_model.to(device)
    bilstm_model.eval()
    with torch.no_grad():
        for index, text in tqdm(enumerate(test_data_loader), desc="Evaluation", total=len(test_data_loader)):
            if not device is None:
                text = text.to(device)
            data[index]["label"] = num2label_dict[bilstm_model(text).argmax().item()]
            json_result = json.dumps(data, ensure_ascii=False)

with open(dump_file_path,'w',encoding="utf-8") as destination:
    destination.write(json_result)

Evaluation: 100%|██████████| 1994/1994 [00:14<00:00, 133.20it/s]
